# 2023_06_15

현재 MA , EWMA , Possion , Noraml 4가지의 transfrom을 적용해서 결과를 확인했다.  
* result folder

> EWMA : http://202.31.200.194:8888/tree/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/EWMA_result

> MA : http://202.31.200.194:8888/tree/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/MA_result

> Possion : http://202.31.200.194:8888/tree/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Possion_result

> Normal : http://202.31.200.194:8888/tree/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Normal_result

최종결과 , 예측값과 실제값의 비교 및 heatmap 비교, quantile 별 비고 및 , 실제 예측값 비교 4가지의 plot들을 각 학습마다 생성함  


### data check
현재 MA transform의 성능이 너무 좋음  
아마 이는 transform을 하면서 실제 사건 발생 이전에 값이 나타나서 예측을 쉽게 만드는 것 같음    
transform의 params마다 실제로 사건 발생 이전에 값이 나타나는지 확인하고  
params별 성능 확인이 필요  

### Metric fix
우선 f1 score를 봤을 때 각 transform마다 f1 score가 다름  
MA같은경우에는 너무 크고 이상함  
아마다 metirc 계산시에 옵션이 이상하게 적용되는거 일수도 있어서 확인이 필요함  


### rounding
Possion이나 Normal 같은 경우에 Recall이 많이 발생함  
이는 모델이 사건 발생을 너무 많이 발생시키는 방향으로 학습을 하고 있음을 나타냄  
이를 조절하기 위해서 후처리로 round을 반올림 하는데 반올림말 0.5 이상의 값에 대해서 round를 진행  
이후 성능 확인  

### Eearly Stopping
현재 일괄적으로 early stopping을 적용하고 있음  
lr가 큰 경우에는 학습이 불안정하므로 early stopping을 적용하는 것이 좋으나
lr이 작은 경우에는 굳이 할 필요가 없음  
이러한 점을 확인해서 다시 학습을 할 필요가 있음  

# Data Check

실제로 모든 데이터에 확인을 하는 것을 힘듬  
이를 파악하기 위해서 푸아송 분포를 이용해서 사건이 어떻게 일어나는지 확인  

사건의 가까운 시간내에 일어날때에는 이전의 값이 남아 있어서 이때문에 미리 사건이 일어날 것이라는 것을 알 수 있음  
이와같이 통계치를 확인하는 것이 중요함 


> notebook : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/BaseModel_Long%20Beach_5_Temporal%20Sparsity_Distiribution.ipynb#

window_size와 transform 했을때의 데이터를 비교 

# Metric fix

> test notebook : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Confusion_matrix_bulid.ipynb

* test notebook 

> MA : http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Confusion_matrix_MA.ipynb

확인해보니 confusion matirx의 xtick과 ytick이 반대로 되어 있었음   

# loss check

> tensorboard notebook : http://202.31.200.70:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/tensorboard_Long%20Beach_5.ipynb

각 transfrom(MA , EWMA , Possion, Noraml)별로 params및 loss에 따른 loss 확인  
* 확인 해보니 lr를 고정한 상태에서 학습을 돌리고 있었음  

하지만 EWMA 및 Possion의 경우에는 train_loss 및 val_loss가 0이여서 확인이 필요함  

# data leakage

현재 val_data까지 포함한 상태로 학습을 했음  
현재 transform 중에서 가장 성능이 좋은 값들과 val_data를 제거한 데이터로 학습을 하고 학습 결과를 비교함  

현재 학습 상태가 제일 좋은 transform별로 params 확인이 필요함  

> http://202.31.200.194:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/Transform_Result_Long%20Beach_5.ipynb

![](../Meeting_Img/2023_06_14_MA.png)
![](../Meeting_Img/2023_06_14_EWMA.png)
![](../Meeting_Img/2023_06_14_Possion.png)
![](../Meeting_Img/2023_06_14_Normal.png)

현재 EMMA 및 Possion은 학습이 안되는 상태이므로 제외하고 테스트 

| transform | params |     lr |
|----------:|--------|-------:|
|        MA | 3      | 0.0005 |
|    Noraml | 1.1143 |   0.01 |
| EWMA      |        |        |
| Possion   |        |        |

다음 상황에서 새로운 데이터로 학습 및 기존의 data leakage 된 결과와 비교  

> tensorboard: http://202.31.200.70:8888/notebooks/NPLAB-NAS/Members/SEO/Emergency_Demand/Traffic_Accient/TFT/tensorboard_Long%20Beach_5.ipynb